In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,71,2024-03-23,-5,111.357143,96.437143,0.533900,10.987143,24.780000,0.178314,114.761429,...,11.175362,29.327536,0.196217,118.039130,0,3,5,2,3,2024
1,70,2024-03-23,-1,117.884058,98.768116,0.560855,11.289855,24.137681,0.173507,118.062319,...,12.748571,25.748571,0.221786,114.528571,0,3,5,2,26,2024
2,70,2024-03-23,-3,106.594203,96.878261,0.525797,12.018841,21.436232,0.165391,109.759420,...,10.785507,27.469565,0.209826,118.636232,0,3,5,4,5,2024
3,71,2024-03-23,2,117.071429,97.811429,0.567900,12.741429,25.157143,0.230600,119.174286,...,12.642029,22.988406,0.174159,111.178261,0,3,5,2,24,2024
4,70,2024-03-23,-8,113.304348,98.573913,0.540043,11.540580,24.571014,0.180855,114.146377,...,11.864286,20.037143,0.172471,111.195714,0,3,5,3,28,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-23,BRK,NYK,3.450000,1.327869,0
1,2024-03-23,SAC,ORL,2.300000,1.645161,0
2,2024-03-23,CHO,ATL,4.100000,1.253165,0
3,2024-03-23,PHO,SAS,1.212766,4.600000,1
4,2024-03-23,TOR,WAS,2.850000,1.444444,0


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-23,BRK,NYK,3.450000,1.327869,0
1,2024-03-23,SAC,ORL,2.300000,1.645161,1
2,2024-03-23,CHO,ATL,4.100000,1.253165,0
3,2024-03-23,PHO,SAS,1.212766,4.600000,1
4,2024-03-23,TOR,WAS,2.850000,1.444444,1


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-23,BRK,NYK,3.450000,1.327869,0
1,2024-03-23,SAC,ORL,2.300000,1.645161,1
2,2024-03-23,CHO,ATL,4.100000,1.253165,0
3,2024-03-23,PHO,SAS,1.212766,4.600000,1
4,2024-03-23,TOR,WAS,2.850000,1.444444,1


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-23,BRK,NYK,3.450000,1.327869,0
1,2024-03-23,SAC,ORL,2.300000,1.645161,0
2,2024-03-23,CHO,ATL,4.100000,1.253165,0
3,2024-03-23,PHO,SAS,1.212766,4.600000,1
4,2024-03-23,TOR,WAS,2.850000,1.444444,0


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-23,BRK,NYK,3.450000,1.327869,0.0
1,2024-03-23,SAC,ORL,2.300000,1.645161,1.0
2,2024-03-23,CHO,ATL,4.100000,1.253165,0.0
3,2024-03-23,PHO,SAS,1.212766,4.600000,1.0
4,2024-03-23,TOR,WAS,2.850000,1.444444,0.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-23,BRK,NYK,3.450000,1.327869,0.0
1,2024-03-23,SAC,ORL,2.300000,1.645161,1.0
2,2024-03-23,CHO,ATL,4.100000,1.253165,0.0
3,2024-03-23,PHO,SAS,1.212766,4.600000,1.0
4,2024-03-23,TOR,WAS,2.850000,1.444444,0.0
